### Training Data

In [14]:
import matplotlib.pyplot as plt

import numpy as np
import numpy
import torch
import time
from timeit import default_timer as timer
import random
from tqdm.autonotebook import tqdm

from mkidreadoutanalysis.quasiparticletimestream import QuasiparticleTimeStream
from data_engineering import make_dataset, plot_stream_data, train_step, test_step, make_predictions

In [15]:
# Generate the dataset

NO_PULSE_FRACTION = 0.0
NUM_SAMPLES = 10000 # This is approximate, the number of photons in the last iteration of the loop is Poisson distributied
QP_TIME_LENGTH = 0.01 # secs
SAMPLING_FREQ = 2e6 # Hz
FALL_TIME = 30
EDGE_PAD = FALL_TIME * 2
WINDOW_SIZE = 1000
RANDOM_SEED = 42


no_pulses = []
pulses = []

# Generate qp timestream object
qp_timestream = QuasiparticleTimeStream(SAMPLING_FREQ, QP_TIME_LENGTH)
qp_timestream.gen_quasiparticle_pulse(tf = FALL_TIME)

# Make the dataset
make_dataset(qp_timestream,
             NUM_SAMPLES,
             NO_PULSE_FRACTION,
             pulses,
             no_pulses,
             True,
             edge_padding=EDGE_PAD,
             window_size=WINDOW_SIZE)


More than 1 photon arriving per time step. Lower the count rate?


Num samples with photons: 10000/10000.0
Number of samples with pulses: 10000
Number of samples without pulses: 0


In [16]:
# Now lets start splitting out the training samples and the label. In this model, we want scalar value for the label, not a time series
X = []
y = []

# Lets create one big list of the pulse and no pulse samples randomly shuffled together 
train_data = pulses + no_pulses
random.shuffle(train_data)

# Now lets separate the training samples (I/Q data) from the label data (photon arrival)
for element in train_data:
    X.append(element[0:2,:])
    y.append(np.argwhere(element[2] == 1))

In [17]:
# With the training and label data now separated, lets start defining our training/testing metrics
# and split the dataset into train and test
TEST_RATIO = 0.2
BATCH_SIZE = 32

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_RATIO, # Ratio of test data to use from full dataset; Training is the complement
    random_state=RANDOM_SEED
)

# Now lets convert the lists to Tensors. Converting to np arrays first based on warning from torch
X_train = torch.Tensor(numpy.array(X_train))
X_test = torch.Tensor(numpy.array(X_test))
y_train = torch.Tensor(numpy.array(y_train))
y_test = torch.Tensor(numpy.array(y_test))

print(f'# of train samples: {len(X_train)}, # of test samples: {len(X_test)}')

# of train samples: 8000, # of test samples: 2000


In [18]:
# It's finally time to create our Dataloader objects
from torch.utils.data import TensorDataset, DataLoader

# Let's first convert from numpy arrays to Tensors and create datasets
train_dataset = TensorDataset(X_train,
                              y_train)
test_dataset = TensorDataset(X_test,
                             y_test)

train_dloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
test_dloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# Now lets inpect the objects.
print(f'Type: {type(train_dloader)}')
train_batch_img, train_batch_labels = next(iter(train_dloader))
print(f'Batch Img: {train_batch_img.shape}, Batch Labels: {train_batch_labels.shape}')

Type: <class 'torch.utils.data.dataloader.DataLoader'>
Batch Img: torch.Size([32, 2, 1000]), Batch Labels: torch.Size([32, 1, 1])


### Model Definition

In [24]:
from models import ConvRegv1

In [25]:
# Lets create a model instance, loss, and optimizer
torch.manual_seed(RANDOM_SEED)

conv_reg_v1 = ConvRegv1(in_channels=2)
optimizer = torch.optim.SGD(params=conv_reg_v1.parameters(), lr=0.1)
loss_fn = torch.nn.L1Loss(reduction='mean')# 'mean' reduction takes all the loss values from the batch and averages them to get the loss

In [26]:
# Now lets give the train/test loop a try!

# Now lets create a quick little function that gives the run time of the loop
total_time = lambda start_time, stop_time: stop_time - start_time

EPOCHS = 100
train_time_cnn_start = timer()
for epoch in tqdm(range(EPOCHS)):
    print(f'Epoch: {epoch}\n-----------')
    train_step(
        conv_reg_v1,
        train_dloader,
        loss_fn,
        optimizer,
    )
    test_step(
        conv_reg_v1,
        test_dloader,
        loss_fn,
    )
train_time_cnn_end = timer()
print(f'Total time to train: {total_time(train_time_cnn_start, train_time_cnn_end):.2f}s')

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 0
-----------
Batches processed: 250/250, Samples processed: 8000/8000
-----------
Mean Train Loss: 32160.5918


  1%|          | 1/100 [00:18<29:53, 18.12s/it]

Mean Test Loss: 228.4267
-----------

Epoch: 1
-----------
Batches processed: 250/250, Samples processed: 8000/8000
-----------
Mean Train Loss: 276.3838


  2%|▏         | 2/100 [00:36<29:48, 18.25s/it]

...

Mean Test Loss: 226.8855
-----------

Epoch: 98
-----------
Batches processed: 250/250, Samples processed: 8000/8000
-----------
Mean Train Loss: 226.1315


 99%|█████████▉| 99/100 [30:18<00:18, 18.47s/it]

Mean Test Loss: 226.6535
-----------

Epoch: 99
-----------
Batches processed: 250/250, Samples processed: 8000/8000
-----------
Mean Train Loss: 225.9870


100%|██████████| 100/100 [30:37<00:00, 18.37s/it]

Mean Test Loss: 226.6019
-----------

Total time to train: 1837.47s


In [27]:
# Pick n random samples/labels from the test data and plot them
test_samples = []
test_labels = []

for sample, label in random.sample(list(test_dataset), k=15): # random.sample samples k elements from the given population without replacement; returns list of samples.
    test_samples.append(sample)
    test_labels.append(label)

print(f'Test Sample Shape: {test_samples[0].shape}, Test Label Shape: {test_labels[0].shape}')
preds = make_predictions(conv_reg_v1, [x.unsqueeze(dim=0) for x in test_samples]) # returns a tensor
print(f'Preds shape {preds[0].shape}')



Test Sample Shape: torch.Size([2, 1000]), Test Label Shape: torch.Size([1, 1])
Preds shape torch.Size([1, 1])


In [ ]:
for i in range(10):
    pred_stream = torch.zeros_like(test_samples[0][0])
    pred_stream[preds[i].int()] = 1
    plot_stream_data(test_samples[i][0].numpy(),
                     test_samples[i][1].numpy(),
                     pred_stream.numpy(),
                     units='us')